![](img/bitcoin_for_hackers.png)

This is a Jupyter notebook. We're going to use it to interact with a running Bitcoin node.

To run the code in a cell, press shift-enter.

In [ ]:
# Run some code
print("2 + 3 = {}".format(2 + 3))

In [ ]:
# Import our dependencies
from ecc import PrivateKey, get_key_pair
import helper

# Digital Signatures

### Signing a message

Digital signing uses a *private* key and a *public* key:
- the *private* key is kept secret, and is used to *sign* a message.

In [ ]:
# get a keypair
privkey, pubkey = get_key_pair()

print("Private key: {}\n".format(privkey))
print("Public key: {}\n".format(pubkey))

# Only someone with the private key can sign a message:
print("Message: 'Bitcoin for Hackers'\n")

message = 'Bitcoin for Hackers'
# convert message to an integer

sig = privkey.sign(message)
print("Signature: {}\n".format(sig))

### Verifying a message

- the *public* key is shared publicly, and is used to *validate* a signed message.

In [ ]:
# use the public key to verify the signature
is_valid = pubkey.verify(message, sig)
print("Signature {}".format("valid" if is_valid else "invalid"))

In [ ]:
# is the signature still valid if we change the message?
is_valid = pubkey.verify('Bitcoin for Hackerz', sig)
print("Signature {}".format("valid" if is_valid else "invalid"))

In [ ]:
# is a different signature valid for the message?
sig.r += 1
is_valid = pubkey.verify(message, sig)
print("Signature {}".format("valid" if is_valid else "invalid"))

# Coins and Transactions

Introducing Alice, Bob and Charlie. Alice starts with one coin. We're going to pass that coin around by signing and validating transactions.

First let's set everything up, and check the balances and transaction history.

In [ ]:
# set up test. Generate keypairs for Alice, Bob and Charlie and print them out.

In [ ]:
# draw balances - this should show a balance of one for Alice and zero for Bob and Charlie

In [ ]:
# print out transaction history - this should show a single transaction

### Sending a transaction

Only Alice knows her private key, so only she can sign a transaction that sends her coin.

Alice can send her coin to Bob.

In [ ]:
tx1 = txs[-1]
tx2 = create_transaction(tx1, Bob.pubkey)
signed_tx2 = Alice.sign(tx2)
submit_transaction(signed_tx2)

In [ ]:
# draw balances - this should show a balance of one for Bob and zero for Alice and Charlie

In [ ]:
# print out transaction history - this should show two transactions

Anyone can verify the transaction with Alice's *public* key

In [ ]:
is_valid = verify_transaction(signed_tx, Alice.pubkey)
print("Transaction {}".format("valid" if is_valid else "invalid"))

Bob can now sign a new transaction, sending his coin to Charlie.

In [ ]:
tx2 = txs[-1]
tx3 = create_transaction(tx1, Charlie.pubkey)
signed_tx3 = Bob.sign(tx3)
submit_transaction(signed_tx3)

In [ ]:
# draw balances - this should show a balance of one for Charlie and zero for Alice and Bob

In [ ]:
# print out transaction history - this should show three transactions

# The Double spend problem

Bob can sign a new transaction, spending the same coins!

In [ ]:
tx2b = txs[-1]
tx3b = create_transaction(tx1, Alice.pubkey)
signed_tx3b = Bob.sign(tx3b)
submit_transaction(signed_tx3b)

In [ ]:
# draw balances - this should show a balance of one for Alice and Charlie and zero for Bob. Ruh roh!

In [ ]:
# print out transaction history - this should show four transactions

### Ordering

Let's change our ledger logic so only the first spend of a coin is valid.

In [ ]:
# draw balances - this should show a balance of one for Charlie and zero for Alice and Bob.

In [ ]:
# print out transaction history - this should show three valid transactions and one invalid transaction

Seems to have fixed things, right?

Not so fast!

On a distributed network, there isn't a *canonical* ordering. What if Bob sends the Alice transaction to Alice, and the Charlie transaction to Charlie. Alice sees her transaction first, so her view of the ledger is as above. Charlie sees his transaction first, so he sees:

In [ ]:
txs[-2], txs[-1] = txs[-1], txs[-2]  # Charlie sees the last two transactions in a different order!
# draw balances - this should show a balance of one for Charlie and zero for Alice and Bob.

In [ ]:
# print out transaction history - this should show three valid transactions and one invalid transaction

# Cryptographic hashes

Cryptographic hashes have the following properties:

1. it is *infeasible* to generate a message from its hash value (preimage resistance)
2. a small change to a message results in a completely different digest (avalanche effect)
3. it is *infeasible* to find two different messages with the same hash value (collision resistance)

Let's test (1) and (2):

- What's the double sha256 digest of "Bitcoin for Hackers"?
- What's the double sha256 digest of "Bitcoin for Hackerz"?
- What's the preimage for the digest "0x707e397fc6c0327b6d9c3a2be68d4fd2456609caa53f8a4fdc7131d3944b516c"?

In [ ]:
def hash(message):
    # hash the block with the nonce
    print("{0:#0{1}x}".format(int.from_bytes(helper.double_sha256((message).encode()), 'big'), 66))

hash(message="Bitcoin for Hackers")

# Proof-of-Work

Proof-of-work determines who gets to build the next block in the Bitcoin blockchain. It relies on the fact that a cryptographic hash function is one-way, and the output is essentially randomly distributed.

We're going to do some manual proof-of-work over a short message.

Find a valid 'block' for the message "Bitcoin for Hackers" with 4 bits of difficulty.

Try running the `validate_block()` function with the same message but different nonces.

In [ ]:
def validate_block(message, nonce, difficulty):
    # hash the block with the nonce
    block_hash = int.from_bytes(helper.double_sha256((message + nonce).encode()), 'big')
    if block_hash < 2 ** (256 - difficulty):
        # winner! Our digest was below the target difficulty
        print("Valid block:    {0:#0{1}x}.\nNonce = {2}".format(block_hash, 66, nonce))
        return True
    else:
        # Sorry, your digest was too large!
        print("Invalid block:  {0:#0{1}x}.\nNonce = {2}".format(block_hash, 66, nonce))
        return False

validate_block(message="Bitcoin for Hackers", nonce="nonce1", difficulty=4)

# Validating work

Proof-of-work is *hard* to do, but *easy* to validate. Once you and your neighbor have a valid block, verify that their block is valid by running `validate_block()` with their nonce.

In [ ]:
validate_block(message="Bitcoin for Hackers", nonce=<put a valid nonce here>, difficulty=4)